In [1]:
from Py_scripts.overlay import FSKOverlay

In [2]:
ol = FSKOverlay(bitfile_name='/home/xilinx/jupyter_notebooks/rfsoc_fsk/Bitstreams/rfsoc_fsk_v7_40.bit', run_test=False, debug_test=False)
#ol = FSKOverlay(bitfile_name='/home/xilinx/jupyter_notebooks/rfsoc_fsk/Bitstreams/rfsoc_fsk_v8_2.bit', run_test=False, debug_test=False)
#ol = FSKOverlay(bitfile_name='/home/xilinx/jupyter_notebooks/rfsoc_fsk/Bitstreams/rfsoc_pfsk_v1_2.bit', run_test=False, debug_test=False)

In [3]:
ol.radio_transmitter.visualise()

Accordion(children=(VBox(children=(Tab(children=(FigureWidget({
    'data': [{'mode': 'markers',
             …

In [4]:
ol.radio_receiver.visualise()

Accordion(children=(VBox(children=(Tab(children=(FigureWidget({
    'data': [{'mode': 'markers',
             …

In [15]:
ol.radio_receiver.controller.freq_offset = 31 # Use 31 in back to back oct24 2:37pm

In [110]:
ol.radio_receiver.controller.threshold = 0x4AAA
#ol.radio_receiver.controller.threshold = 0x24B7 # Use this in back to back oct24 2:37pm
#ol.radio_receiver.controller.threshold = 0x2000
#ol.radio_receiver.controller.threshold = 0x27B0

In [107]:
ol.radio_transmitter.controller.enable_data = 0
ol.radio_transmitter.controller.enable_transmitter = 0
ol.radio_receiver.controller.reset_time_sync = 1
ol.radio_receiver.controller.reset_phase_sync = 1

In [108]:
ol.radio_receiver.controller.reset_time_sync = 0
ol.radio_receiver.controller.reset_phase_sync = 0
ol.radio_transmitter.controller.enable_data = 0
ol.radio_transmitter.controller.enable_transmitter = 1

In [109]:
ol.radio_transmitter.controller.enable_data = 1
ol.radio_transmitter.controller.enable_transmitter = 1

In [27]:
ol.radio_receiver.controller.freq_offset = 0

In [23]:
ol.radio_receiver.controller.freq_offset

-0.0

In [ ]:
import time
while 1:
    ol.radio_transmitter.controller.enable_data = 0
    ol.radio_transmitter.controller.enable_transmitter = 0
    time.sleep(0.000005)
    ol.radio_transmitter.controller.enable_data = 1
    ol.radio_transmitter.controller.enable_transmitter = 1

In [10]:
ol.radio_receiver.terminal()

Accordion(children=(HBox(children=(VBox(children=(Textarea(value='', disabled=True, layout=Layout(height='200p…

In [11]:
ol.radio_transmitter.data('Hello World!\r')
ol.radio_transmitter.start()

In [12]:
ol.radio_transmitter.data(''.join(['The quick brown fox jumps over the lazy dog.\r',
                             'How razorback-jumping frogs can level six piqued gymnasts.\r']))
ol.radio_transmitter.start()

In [15]:
from pynq import allocate
import numpy as np
from random import randint

frame_size = 44
random_size = 10
        
_flags = 0
_frame_number = 0
data=np.array([72, 101, 108, 108, 111,  32,  87, 111, 114, 108, 100,  33])
dtype=np.uint8 
eof=1
padding=0
msg = np.array(data, dtype=np.uint8)
# Append Barker and Random Data
bkr = np.array([0, 0, 63, 112, 28, len(msg) + 5, _frame_number, _flags, 5, len(msg), padding], dtype=np.uint8)
rnd = np.array([randint(0, 255) for p in range(0, random_size)], dtype=np.uint8)
seq = np.append(bkr, msg)
seq = np.append(rnd, seq)
pad = np.append(seq, np.zeros(int(np.ceil((len(rnd) + len(bkr) + len(msg))/32) * 32 - (len(rnd) + len(bkr) + len(msg))), dtype=np.uint8))
txbuf = allocate(shape=(len(pad),), dtype=np.uint8)
txbuf[:] = pad[:]
print(txbuf)
rxbuf = allocate(shape=(len(txbuf),), dtype = np.uint8)

[185  81 189 125  84  41 219 191  84  98   0   0  63 112  28  17   0   0
   5  12   0  72 101 108 108 111  32  87 111 114 108 100  33   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [16]:
ol.radio_transmitter.axi_dma.sendchannel.transfer(txbuf)
ol.radio_transmitter.axi_dma.sendchannel.wait()

ol.radio_receiver.axi_dma.recvchannel.transfer(rxbuf)
ol.radio_receiver.controller.transfer = 1
ol.radio_receiver.axi_dma.recvchannel.wait()
ol.radio_receiver.controller.transfer = 0

RuntimeError: DMA channel not started

In [12]:
print(rxbuf)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [15]:
#ol.radio_receiver.controller.receive_size = 32
#buff_len = ol.radio_receiver.controller.receive_size
#ol.radio_receiver._rx_buff.freebuffer()

print(rxbuf)
ol.radio_transmitter.axi_dma.sendchannel.wait()
ol.radio_receiver.controller.transfer = 1
ol.radio_receiver.axi_dma.recvchannel.wait()
ol.radio_receiver.controller.transfer = 0
print(rxbuf)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


RuntimeError: DMA channel not started

In [10]:
# Exactly as in scrpits
from pynq import allocate
import numpy as np
from random import randint

frame_size = 44
random_size = 10
        
_flags = 0
_frame_number = 0
data=np.array([72, 101, 108, 108, 111,  32,  87, 111, 114, 108, 100,  33])
dtype=np.uint8 
eof=1
padding=0
msg = np.array(data, dtype=np.uint8)
# Append Barker and Random Data
bkr = np.array([0, 0, 63, 112, 28, len(msg) + 5, _frame_number, _flags, 5, len(msg), padding], dtype=np.uint8)
rnd = np.array([randint(0, 255) for p in range(0, random_size)], dtype=np.uint8)
seq = np.append(bkr, msg)
seq = np.append(rnd, seq)
pad = np.append(seq, np.zeros(int(np.ceil((len(rnd) + len(bkr) + len(msg))/32) * 32 - (len(rnd) + len(bkr) + len(msg))), dtype=np.uint8))
txbuf = allocate(shape=(len(pad),), dtype=np.uint8)
txbuf[:] = pad[:]
print(txbuf)
rxbuf = allocate(shape=(len(txbuf),), dtype = np.uint8)

[ 60  78  87  49 226 141  18 181  35  58   0   0  63 112  28  17   0   0
   5  12   0  72 101 108 108 111  32  87 111 114 108 100  33   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [11]:
ol.radio_receiver.controller.transfer = 0
ol.radio_receiver.controller.transfer = 1
ol.radio_transmitter.axi_dma.sendchannel.transfer(txbuf)
ol.radio_receiver.axi_dma.recvchannel.transfer(rxbuf)



In [12]:
rxbuf

PynqBuffer([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0], dtype=uint8)

In [13]:
ol.radio_transmitter.axi_dma.sendchannel.wait()

In [ ]:
ol.radio_receiver.axi_dma.recvchannel.wait()

In [ ]:
ol.radio_receiver.controller.transfer = 0

In [82]:
int(0x2DA0)

11680